In [6]:
import numpy as np
from scipy.special import eval_hermite, eval_genlaguerre, j1, genlaguerre
from scipy.optimize import newton, root
import imageio as iio
from PIL import Image
import os
import math

# Define Hermite and Laguerre polynomials
def hermite(n, x):
    return eval_hermite(n, x)

#屏幕参数

#激光参数
z = 0
lamda=0.66e-9
w0=500e-6
zr = np.pi * w0**2 / lamda
wz = w0 * np.sqrt(1 + (z / zr)**2)
k0 = 2 * np.pi / lamda
# Function to generate phase pattern
def generate_phase_pattern(mode='LG', mx=1152, my=1920, a_scale=5, kx=1, d=4e-6, s=4e-6):
    pix_size = 9.2e-6
    x = np.linspace(-mx*pix_size/2, mx*pix_size/2, mx)
    y = np.linspace(-my*pix_size/2, my*pix_size/2, my)
    X, Y = np.meshgrid(x, y)
    r = np.sqrt(X**2 + Y**2)
    theta = np.arctan2(Y, X)
    if mode == 'LG*':
        l = 1
        p = 0
        L = abs(l)
        phase = np.exp(-1j * l * theta)
        am = (2 * math.factorial(p) / (np.pi * math.factorial(p + L)))**0.5 * \
             (np.sqrt(2) * r / w0)**L * np.exp(-(r**2 / w0**2)) * \
             genlaguerre(p, L)(2 * r**2 / w0**2)
    elif mode == 'HG*':
        l = 1
        p = 0
        phase = np.exp(-(X**2 + Y**2) / w0**2) 
        am = hermite(l, np.sqrt(2) * X / w0) * hermite(p, np.sqrt(2) * Y / w0) * \
             (1 / w0) * (2 / (np.pi * 2**(l + p) * math.factorial(p) * math.factorial(l)))**0.5
    elif mode == 'LG00*':
        l = 0
        p = 0
        L = abs(l)
        phase = np.exp(-1j * l * theta)
        am = (2 * math.factorial(p) / (np.pi * math.factorial(p + L)))**0.5 * \
             (np.sqrt(2) * r / w0)**L * np.exp(-(r**2 / w0**2)) * \
             genlaguerre(p, L)(2 * r**2 / w0**2)
    
    u = am * phase
    AM = np.abs(u)/np.max(np.abs(u)) 
    phi = np.angle(u)
    
    # Add grating phase
    #gra = 2 * np.pi * X / lamda 
    grating_period= pix_size * 5
    gra = (2 *  np.pi * X / grating_period) % (2 * np.pi)
    phi = phi + gra - np.pi
    # Method F
    C = 0.5818

    def faeq(f_a_ref, a_ref):
        global C
        return j1(f_a_ref) - C * a_ref

    def find_f_a_ref(a_ref):
        return newton(faeq, np.ones_like(a_ref) * 1, args=(a_ref,))
    fa = find_f_a_ref(AM)
    h = fa * np.sin(phi)
    h = np.mod(h + np.min(h), 2 * np.pi)
    
    # Create a full black image and place the phase pattern in the center
    gh = (h * 255 / (2 * np.pi)).astype(np.uint8)
    return gh

# Output directory
output_dir = r'C:\Users\11455\Desktop\wkx1\程序\python\相图'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Generate LG mode phase pattern
lg_phase = generate_phase_pattern('LG*')
lg_image_path = os.path.join(output_dir, 'lg_phase.bmp')
iio.imwrite(lg_image_path, lg_phase)

# Generate HG mode phase pattern
hg_phase = generate_phase_pattern('HG*')
hg_image_path = os.path.join(output_dir, 'hg_phase.bmp')
iio.imwrite(hg_image_path, hg_phase)

# Generate LG00 mode phase pattern
gs_phase = generate_phase_pattern('LG00*')
gs_image_path = os.path.join(output_dir, 'gs_phase.bmp')
iio.imwrite(gs_image_path, gs_phase)

print(f"Images saved to {output_dir}")


NameError: name 'C' is not defined